In [215]:
from IPython.core.display import display, HTML
display(HTML('<style>.container {width:1260px !important;}</style>'))

<h1>KBO 데이터 크롤링 및 메이저리그로 진출할만한 선수 찾아보기</h1>
<h2>1. <a href='https://www.koreabaseball.com/Default.aspx'>KBO</a> 크롤링 및 데이터 전처리</h2>
<h2>2. 2020년 기준 KBO 상위 10명의 선수와 한국인 메이저리거 선수들 기록 분석통해 메이저리그로 갈만한 선수 찾아보기</h2>

---

<h3>한국인 메이저리거 데이터</h3>

In [216]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [217]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

In [218]:
# get korean majorleaguer from namuwiki
url = 'https://namu.wiki/w/%ED%95%9C%EA%B5%AD%EC%9D%B8%20%EB%A9%94%EC%9D%B4%EC%A0%80%EB%A6%AC%EA%B1%B0'
req = requests.get(url)
html = req.text
soup = bs(html, 'html.parser')

In [219]:
korean_majorleaguer = []

for td in soup('td'):
    for a in td('a'):
        if ('class' in a.attrs
            and 'wiki-link-internal' in a['class']
            and len(a.get_text())==3):
            
            korean_majorleaguer.append(a.get_text())
            
korean_majorleaguer = sorted(list(set(korean_majorleaguer)))

In [220]:
player_position = ['내야수', '투수', '투수', '투수', '투수', '내야수', '외야수', '투수', 
                   '투수', '내야수', '투수', '투수', '투수', '투수', '투수', '내야수',
                  '투수', '투수', '투수', '내야수', '내야수', '외야수', '내야수']
player_birth_year = ['1987', '1969', '1988', '1979', '1977', '1995', '1988', '1983', '1987', '1986', '1973', '1980',
                    '1980', '1977', '1982', '1982', '1971', '1976', '1975', '1991', '1979', '1982', '1987']
korean_majorleaguer_df = pd.DataFrame(data={'Name': korean_majorleaguer, 'Birth_year': player_birth_year,
                                            'Position': player_position})
korean_majorleaguer_df.head()

,Name,Birth_year,Position
0,강정호,1987,내야수
1,구대성,1969,투수
2,김광현,1988,투수
3,김병현,1979,투수
4,김선우,1977,투수


<h3>한국인 메이저리거 url data</h3>

In [221]:
from selenium import webdriver
import time

In [222]:
# crawling using selenium(too slow...)

# player_info_url = []

# # execute chrome web browser
# path = '/Users/dohee/Downloads/chromedriver'

# for player in korean_majorleaguer_df.loc[:]['Name'].to_list():
#     driver = webdriver.Chrome(path)
#     driver.get('https://www.koreabaseball.com/Player/Search.aspx')
    
#     # search player
#     element = driver.find_element_by_id("cphContents_cphContents_cphContents_txtSearchPlayerName")
#     element.send_keys(player)

#     # click the search button
#     driver.find_element_by_xpath('//*[@id="cphContents_cphContents_cphContents_btnSearch"]').click()
#     time.sleep(2)

#     # count number of person who has same name
#     count_search_result = driver.find_element_by_xpath('//*[@id="cphContents_cphContents_cphContents_udpRecord"]/div[2]/p/strong/span')
    
#     # if there's no information of the player
#     if count_search_result.text == '0':
#         player_info_url.append('NO_INFO_URL')
#     # if there's only one information of the player
#     elif count_search_result.text == '1':
#         driver.find_element_by_xpath('//*[@id="cphContents_cphContents_cphContents_udpRecord"]/div[2]/table/tbody/tr/td[2]/a').click()
#         player_info_url.append(driver.current_url)
#     # if there are multiple information of the player
#     else:
#         temp_ls = []
#         for count in range(int(count_search_result.text)):
#             driver.find_element_by_xpath('//*[@id="cphContents_cphContents_cphContents_udpRecord"]/div[2]/table/tbody/tr[{0}]/td[2]/a'.format(count+1)).click()
#             temp_ls.append(driver.current_url)
#             time.sleep(2)
#             driver.back()
#             time.sleep(2)
            
#             # search player
#             element = driver.find_element_by_id("cphContents_cphContents_cphContents_txtSearchPlayerName")
#             element.send_keys(player)

#             # click the search button
#             driver.find_element_by_xpath('//*[@id="cphContents_cphContents_cphContents_btnSearch"]').click()
#             time.sleep(2)
            
#         player_info_url.append(temp_ls)
#     driver.close()
    
# # add to dataframe
# korean_majorleaguer_df['info_url'] = player_info_url

In [223]:
# crwaling using 'post' method (more faster than selenium)

import re
base_url = 'https://www.koreabaseball.com'
player_info_url = []

for idx, player_name in enumerate(korean_majorleaguer_df.loc[:]['Name'].to_list()):
    url = 'https://www.koreabaseball.com/Player/Search.aspx'
    _headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36'}
    _data = {'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
            'ctl00$ctl00$ctl00$cphContents$cphContents$ScriptManager1': 'ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$udpRecord|ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$btnSearch',
    '__VIEWSTATE': '/wEPDwUJMTMxNzk5NDM2D2QWAmYPZBYCZg9kFgJmD2QWAgIBD2QWAmYPZBYCAgEPZBYCAgUPZBYCAgEPZBYCZg9kFghmDxAPFgYeDURhdGFUZXh0RmllbGQFCEZJUlNUX05NHg5EYXRhVmFsdWVGaWVsZAUEVF9JRB4LXyFEYXRhQm91bmRnZBAVCwrtjIAg7ISg7YOdAk5DBuuRkOyCsAJLVAJMRwbtgqTsm4ADS0lBBuuhr+uNsAbsgrzshLECU0sG7ZWc7ZmUFQsAAk5DAk9CAktUAkxHAldPAkhUAkxUAlNTAlNLAkhIFCsDC2dnZ2dnZ2dnZ2dnFgFmZAIBDxBkZBYBZmQCAg8PZBYCHgpvbmtleXByZXNzBXlpZihldmVudC5rZXlDb2RlID09IDEzKXtfX2RvUG9zdEJhY2soJ2N0bDAwJGN0bDAwJGN0bDAwJGNwaENvbnRlbnRzJGNwaENvbnRlbnRzJGNwaENvbnRlbnRzJGJ0blNlYXJjaCcsJycpO3JldHVybiBmYWxzZTt9ZAIFDw8WBh4JUGFnZUluZGV4BQExHghQYWdlU2l6ZQUCMjAeDVRvdGFsUm93Q291bnRmZBYcAgEPDxYCHgdWaXNpYmxlaGRkAgMPDxYCHwdoZGQCBQ8PFgYeBFRleHQFATEeCENzc0NsYXNzBQJvbh4EXyFTQgICZGQCBw8PFggfCAUBMh8JZR8KAgIfB2hkZAIJDw8WCB8IBQEzHwllHwoCAh8HaGRkAgsPDxYIHwgFATQfCWUfCgICHwdoZGQCDQ8PFggfCAUBNR8JZR8KAgIfB2hkZAIPDw8WBh8JZR8KAgIfB2hkZAIRDw8WBh8JZR8KAgIfB2hkZAITDw8WBh8JZR8KAgIfB2hkZAIVDw8WBh8JZR8KAgIfB2hkZAIXDw8WBh8JZR8KAgIfB2hkZAIZDw8WAh8HaGRkAhsPDxYCHwdoZGRk1A6xuCh0ccetMvmoHty6C+iZPJRR4TUTclPN2WV25KM=',
    '__VIEWSTATEGENERATOR': '6942A5F7',
    '__EVENTVALIDATION': '/wEdABV25zroy0y+ySxAb8qg0Lh+qUK6+mRsdxhnNCA+WG212kNgeBSbNKLKcKTg82HGZ5MBG/Pf3I75C3rKX8xVtEwo+kQ8AqPLE4RrhO6ZFCqeBK7PzKYNHJn1ix41G9IhOQuJXHqRUXRUZklBZmuFWpsNdCA+bt9lrvQ7Bnt3sOnVYKuMptg8kI77SbU4++55lNksVPJEqMmnsPvHB9dIJGpf5hXVFrJQTuW7aWcjKnYReCWPyJkVhvXuOBYs5qqOoClVZUXxd/uiJhLNEa+9XxmnYsasFt+7FfwAs6g8a36bapjfFDeu2NDc13pCkdcMGuGM1LdB1BdIShiN9H1skvV4F1JgdjTRh5oeAN6Yg5M8ELpmrE3ZItDca36f0/kynHQIcBFwjnAKA7Co0KA6ls0JuuwNcLXb9Hf3j9q4Bnx0xSIEsp38v5tQJGTakYgKU9EJBIfUBLFxAijQLbL4FCib7tDqjkD5YMtH7xvNaxZaMg==',
    'ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$hfPage': 1,
    'ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$txtSearchPlayerName': '{0}'.format(player_name),
    '__ASYNCPOST': 'true',
    'ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$btnSearch': '검색'}

    html = requests.post(url, data=_data, headers=_headers)
    soup = bs(html.text, 'html.parser')

    html_ls = []
    url_ls = []

    for td in soup.find_all('td'):
        # string between <td>(head) and </td>(tail)
        html_ls.append(str(td)[4:-5])
        if str(td)[4:8] == korean_majorleaguer_df.loc[idx]['Birth_year']:
            url_ls.append(base_url + re.findall(r'\"(.+?)\"', html_ls[-4])[0])

    player_info_url.append(url_ls)
    
korean_majorleaguer_df['info_url'] = player_info_url
korean_majorleaguer_df.head()

,Name,Birth_year,Position,info_url
0,강정호,1987,내야수,[https://www.koreabaseball.com/Record/Retire/H...
1,구대성,1969,투수,[https://www.koreabaseball.com/Record/Retire/P...
2,김광현,1988,투수,[https://www.koreabaseball.com/Record/Retire/P...
3,김병현,1979,투수,[https://www.koreabaseball.com/Record/Retire/P...
4,김선우,1977,투수,[https://www.koreabaseball.com/Record/Retire/P...


In [224]:
# eliminate same name player
for idx, name, url in zip(korean_majorleaguer_df.index,
                          korean_majorleaguer_df.loc[:]['Name'].to_list(), 
                          korean_majorleaguer_df.loc[:]['info_url'].to_list()):
    if len(url) >= 2:
        print("url 여러 개인 선수: ", name, ' ', ', index =', idx)

url 여러 개인 선수:  이상훈   , index = 16


In [225]:
print(korean_majorleaguer_df.loc[16]['info_url'][:])
# 두 개의 url 확인 결과 두 번째 url이 메이저리거 이상훈 선수에 대한 데이터였다.

a = korean_majorleaguer_df.loc[16]['info_url'][1]
print([a])
korean_majorleaguer_df['info_url'][16] = [korean_majorleaguer_df.loc[16]['info_url'][1]]

['https://www.koreabaseball.com/Record/Retire/Pitcher.aspx?playerId=90851', 'https://www.koreabaseball.com/Record/Retire/Pitcher.aspx?playerId=93147']
['https://www.koreabaseball.com/Record/Retire/Pitcher.aspx?playerId=93147']


In [226]:
for index in korean_majorleaguer_df.index:
    if korean_majorleaguer_df.loc[index]['info_url']:
        korean_majorleaguer_df['info_url'][index] = korean_majorleaguer_df.loc[index]['info_url'][0]
    else:
        korean_majorleaguer_df['info_url'][index] = ''

In [227]:
for index in korean_majorleaguer_df.index:
    print(korean_majorleaguer_df.loc[index]['Name'], korean_majorleaguer_df.loc[index]['info_url'])

강정호 https://www.koreabaseball.com/Record/Retire/Hitter.aspx?playerId=76325
구대성 https://www.koreabaseball.com/Record/Retire/Pitcher.aspx?playerId=93715
김광현 https://www.koreabaseball.com/Record/Retire/Pitcher.aspx?playerId=77829
김병현 https://www.koreabaseball.com/Record/Retire/Pitcher.aspx?playerId=62349
김선우 https://www.koreabaseball.com/Record/Retire/Pitcher.aspx?playerId=78232
김하성 https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=64300
김현수 https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=76290
류제국 https://www.koreabaseball.com/Record/Retire/Pitcher.aspx?playerId=63111
류현진 https://www.koreabaseball.com/Record/Retire/Pitcher.aspx?playerId=76715
박병호 https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=75125
박찬호 https://www.koreabaseball.com/Record/Retire/Pitcher.aspx?playerId=62761
백차승 
봉중근 https://www.koreabaseball.com/Record/Retire/Pitcher.aspx?playerId=77147
서재응 https://www.koreabaseball.com/Record/Retire/Pit

In [228]:
korean_majorleaguer_dict = {}

for idx, url in enumerate(korean_majorleaguer_df.loc[:]['info_url']):
    check_retired = True
    #print(url, idx)
    
    # if there's an url
    if url:
        if ('HitterDetail' in url) or ('PitcherDetail' in url):
            url = url.replace('Basic', 'Total')
            check_retired = False
            
        req = requests.get(url)
        html = req.text
        soup = bs(html, 'html.parser')

        # gather annual data without career data
        player_temp_table = soup.find('table', {'class': 'tData01 tt mb5'}) if check_retired else soup.find('table', {'class': 'tbl tt mb5'})
        player_column_list = ['연도', '팀명']
        
        try:
            player_column_list_tag = player_temp_table.find_all('th')
        except:
            player_temp_table = soup.find('table', {'class': 'tbl tt mgb5'})
            player_column_list_tag = player_temp_table.find_all('th')

        for col in player_column_list_tag:
            player_a_tag = col.find_all('a')
            for a in player_a_tag:
                player_column_list.append(a.get('title'))

        temp_data = pd.DataFrame(columns=player_column_list)

        i = 0
        index = 0
        col_len = len(player_column_list)

        while True:
            try:
                temp_data.loc[i] = [x.text for x in player_temp_table.find_all('td')[index : index + col_len]]
                i += 1
                index += col_len
            except:
                break

        # gather only career data
        player_temp_table = soup.find('tfoot', {'class': 'play_record'})
        player_column_tag = player_temp_table.find_all('th')

        career = [x.text for x in player_column_tag]
        career.insert(0, '통산')
        temp_data.loc[i] = career

        korean_majorleaguer_dict[korean_majorleaguer_df.loc[idx]['Name']] = temp_data
        
    # if there's no valid url
    else:
        korean_majorleaguer_dict[korean_majorleaguer_df.loc[idx]['Name']] = ''

<h3>한국인 메이저리거 데이터 중 '타자' url 데이터 </h3>

In [232]:
hitter_majorleaguer = korean_majorleaguer_df[korean_majorleaguer_df['Position'] != '투수']
hitter_majorleaguer = hitter_majorleaguer[hitter_majorleaguer['info_url'] != ''].reset_index(drop=True)
hitter_majorleaguer

,Name,Birth_year,Position,info_url
0,강정호,1987,내야수,https://www.koreabaseball.com/Record/Retire/Hi...
1,김하성,1995,내야수,https://www.koreabaseball.com/Record/Player/Hi...
2,김현수,1988,외야수,https://www.koreabaseball.com/Record/Player/Hi...
3,박병호,1986,내야수,https://www.koreabaseball.com/Record/Player/Hi...
4,이대호,1982,내야수,https://www.koreabaseball.com/Record/Player/Hi...
5,최희섭,1979,내야수,https://www.koreabaseball.com/Record/Retire/Hi...
6,황재균,1987,내야수,https://www.koreabaseball.com/Record/Player/Hi...


<h3>2020년 KBO top10 타자 url 데이터</h3>

In [233]:
# get 2020 top10 data of hitter in KBO

base_url = 'https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx'
req = requests.get(base_url)
html = req.text
soup = bs(html, 'html.parser')
top10_hitter_url_df = pd.DataFrame(columns=['Name', 'info_url'])
count = 0

top10_hitter_temp_table = soup.find('table', {'class': 'tData01 tt'})
for url in top10_hitter_temp_table.find_all('a'):
    temp_tag = url.get('href')
    if 'playerId' in temp_tag and count < 10:
        top10_hitter_url_df.loc[count] = [url.text, 'https://www.koreabaseball.com'+url.get('href')]
        count += 1
        
top10_hitter_url_df

,Name,info_url
0,최형우,https://www.koreabaseball.com/Record/Player/Hi...
1,손아섭,https://www.koreabaseball.com/Record/Player/Hi...
2,로하스,https://www.koreabaseball.com/Record/Player/Hi...
3,박민우,https://www.koreabaseball.com/Record/Player/Hi...
4,페르난데스,https://www.koreabaseball.com/Record/Player/Hi...
5,이정후,https://www.koreabaseball.com/Record/Player/Hi...
6,허경민,https://www.koreabaseball.com/Record/Player/Hi...
7,김현수,https://www.koreabaseball.com/Record/Player/Hi...
8,강백호,https://www.koreabaseball.com/Record/Player/Hi...
9,양의지,https://www.koreabaseball.com/Record/Player/Hi...


In [235]:
# 2020 KBO top10 hitter data crawling

top10_hitter_dict = {}

for idx, url in enumerate(top10_hitter_url_df.loc[:]['info_url']):
    url = url.replace('Basic', 'Total')
    req = requests.get(url)
    html = req.text
    soup = bs(html, 'html.parser')
    
    top10_hitter_table = soup.find('table', {'class': 'tbl tt mb5'})
    top10_hitter_column_list = ['연도', '팀명']
    
    for col in top10_hitter_table.find_all('th'):
        for a in col.find_all('a'):
            top10_hitter_column_list.append(a.get('title'))
            
    temp_data = pd.DataFrame(columns=top10_hitter_column_list)
    
    i = 0
    index = 0
    
    while True:
        try:
            temp_data.loc[i] = [x.text for x in top10_hitter_table.find_all('td')[index : index + len(top10_hitter_column_list)]]
            i += 1
            index += len(top10_hitter_column_list)
        except:
            break
            
    top10_hitter_table = soup.find('tfoot', {'class': 'play_record'})
    
    career = [x.text for x in top10_hitter_table.find_all('th')]
    career.insert(0, '통산')
    temp_data.loc[i] = career
            
    top10_hitter_dict[top10_hitter_url_df.loc[idx]['Name']] = temp_data

In [236]:
majorleaguer_avg = []
top10_hitter_avg = []

for col in korean_majorleaguer_dict['강정호'].columns[2:]:
    if col != '타율':
        temp_majorleaguer_ls = [float(korean_majorleaguer_dict[name].iloc[korean_majorleaguer_dict[name].shape[0]-1][col])/float(korean_majorleaguer_dict[name].iloc[korean_majorleaguer_dict[name].shape[0]-1]['경기']) for name in hitter_majorleaguer['Name']]
        temp_majorleaguer_np = np.array(temp_majorleaguer_ls)
        majorleaguer_avg.append(round(temp_majorleaguer_np.mean(), 2))
    else:
        temp_majorleaguer_ls = [float(korean_majorleaguer_dict[name].iloc[korean_majorleaguer_dict[name].shape[0]-1][col]) for name in hitter_majorleaguer['Name']]
        temp_majorleaguer_np = np.array(temp_majorleaguer_ls)
        majorleaguer_avg.append(round(temp_majorleaguer_np.mean(), 2))
    
for col in top10_hitter_dict['최형우'].columns[2:]:
    if col != '타율':
        temp_top10_hitter_ls = [float(top10_hitter_dict[name].iloc[top10_hitter_dict[name].shape[0]-1][col])/float(top10_hitter_dict[name].iloc[top10_hitter_dict[name].shape[0]-1]['경기']) for name in top10_hitter_url_df['Name']]
        temp_top10_hitter_np = np.array(temp_top10_hitter_ls)
        top10_hitter_avg.append(round(temp_top10_hitter_np.mean(), 2))
    else:
        temp_top10_hitter_ls = [float(top10_hitter_dict[name].iloc[top10_hitter_dict[name].shape[0]-1][col]) for name in top10_hitter_url_df['Name']]
        temp_top10_hitter_np = np.array(temp_top10_hitter_ls)
        top10_hitter_avg.append(round(temp_top10_hitter_np.mean(), 2))

# print()    
# print(top10_hitter_dict['최형우'].columns)

theta_ = top10_hitter_dict['최형우'].columns[6:-3].to_list()
theta_.insert(len(theta_), top10_hitter_dict['최형우'].columns[-1])
r_ = top10_hitter_avg[4:-3]
r_.insert(len(r_), top10_hitter_avg[-1])

fig = go.Figure()
fig.add_trace(go.Scatterpolar(r=majorleaguer_avg[3:],
                              theta=korean_majorleaguer_dict['강정호'].columns[5:],
                              name='한국인 메이저리거의 KBO시절 평균',
                              marker_color='blue'))
fig.add_trace(go.Scatterpolar(r=r_,
                             theta=theta_,
                             name='2020년 KBO 상위10명의 타자 통산평균',
                             marker_color='crimson'))

fig.update_layout(template=None,
                  paper_bgcolor='rgb(245, 245, 245)',
                  polar=dict(radialaxis=dict(visible=True, linewidth=1, linecolor='Black', ticks=''),
                            angularaxis=dict(showline=True, linewidth=1, linecolor='black', ticks='')))
fig.show()

In [268]:
fig = make_subplots(rows=1, cols=2)

fig.add_trace(go.Bar(x=korean_majorleaguer_dict['강정호'].columns[5:],
                    y=majorleaguer_avg[3:],
                    name='한국인 메이저리거의 KBO시절 통산평균',
                    marker_color='blue'), 1, 2)
fig.add_trace(go.Bar(x=theta_,
                    y=r_,
                    name='2020년 KBO 상위10명 통산평균',
                    marker_color='red'), 1, 2)
fig.add_trace(go.Bar(x=[korean_majorleaguer_dict['강정호'].columns[2]],
                    y=[majorleaguer_avg[0]],
                    marker_color='blue',
                    showlegend=False), 1, 1)
fig.add_trace(go.Bar(x=[korean_majorleaguer_dict['강정호'].columns[2]],
                    y=[top10_hitter_avg[0]],
                    marker_color='red',
                    showlegend=False), 1, 1)

fig.update_yaxes(showgrid=False, zeroline=False, row=1, col=1)
fig.update_yaxes(showgrid=False, zeroline=False, row=1, col=2)

fig.update_layout(legend=dict(orientation='v',
                             yanchor='top', xanchor='right',
                             y=0.79, x=0.998),
                  plot_bgcolor='rgb(245, 245, 245)',
                 bargap=0.3)
fig.show()

- <h3>한국인 메이저리거들의 KBO 선수시절 평균타율이 2020년 KBO 상위 10명 평균타율보다 낮았다.</h3>

In [269]:
hitter_majorleaguer_df_by_career = pd.DataFrame(columns=['이름', '타율'])
top10_hitter_df_by_career = pd.DataFrame(columns=['이름', '타율'])

for idx, name in enumerate(hitter_majorleaguer['Name']):
    #print(korean_majorleaguer_dict[name])
    temp_data = [name, float(korean_majorleaguer_dict[name].iloc[korean_majorleaguer_dict[name].shape[0]-1]['타율'])]
    hitter_majorleaguer_df_by_career.loc[idx] = temp_data
    
for idx, name in enumerate(top10_hitter_dict.keys()):
    temp_data = [name, float(top10_hitter_dict[name].iloc[top10_hitter_dict[name].shape[0]-1]['타율'])]
    top10_hitter_df_by_career.loc[idx] = temp_data
    
hovertemplate_ = hitter_majorleaguer_df_by_career['이름'].to_list()

fig = go.Figure()
fig.add_trace(go.Box(y=hitter_majorleaguer_df_by_career['타율'],
                     boxpoints='all',
                    name='한국인 메이저리거의 KBO 시절 통산타율',
                     text=['이름: {}<br>타율: {}'.format(x, y) for x, y in zip(hitter_majorleaguer_df_by_career['이름'],
                                                                           hitter_majorleaguer_df_by_career['타율'])]))
fig.add_trace(go.Box(y=top10_hitter_df_by_career['타율'],
                     boxpoints='all',
                    name='2020년 상위 10명의 타자 통산타율',
                    text=['이름: {}<br>타율: {}'.format(x, y) for x, y in zip(top10_hitter_df_by_career['이름'],
                                                                          top10_hitter_df_by_career['타율'])]))

fig.update_yaxes(showgrid=False, zeroline=False)
fig.update_layout(plot_bgcolor='rgb(245, 245, 245)')
fig.show()

In [272]:
x_data = []
y_data = []

fig = go.Figure()

for idx, name in enumerate(hitter_majorleaguer['Name']):
    fig.add_trace(go.Scatter(x=korean_majorleaguer_dict[name].iloc[:-1]['연도'].astype(int),
                             y=korean_majorleaguer_dict[name].iloc[:-1]['타율'].astype(float),
                             mode='markers',
                             marker_color='blue',
                            name='{}'.format(name)))
    x_data.extend(korean_majorleaguer_dict[name].iloc[:-1]['연도'].astype(float).to_list())
    y_data.extend(korean_majorleaguer_dict[name].iloc[:-1]['타율'].astype(float).to_list())
    
for idx, name in enumerate(top10_hitter_dict.keys()):
    fig.add_trace(go.Scatter(x=top10_hitter_dict[name].iloc[:-1]['연도'].astype(int),
                            y=top10_hitter_dict[name].iloc[:-1]['타율'].astype(float),
                            mode='markers',
                            marker_color='red',
                            name='{}'.format(name)))
    
fig.update_yaxes(showgrid=False, zeroline=False)
fig.update_xaxes(showgrid=False, zeroline=False)
fig.update_layout(showlegend=False,
                 plot_bgcolor='rgb(245, 245, 245)')
fig.show()

- <h3>타율에서는 메이저리거와 2020년 KBO 상위 10명의 타자 사이에서 뚜렷한 차이나 경향이 잘 파악되지 않는다...</h3>

- <h3>'스마트 배이스볼'이라는 책에서 타율이 잘못된 지표로 심각한 결함을 가지고 있다고 하였는데 그 이유가 19세기에 타율이라는 용어가 만들어졌는데 그 당시 타자는 자신이 원하는 위치에 공을 던져달라고 요구할 수 있었고, 볼넷이 볼 네 개, 그리고 삼진이 스트라이크 세 개가 아니었던적도(1887년에는 스트라이크 네 개를 잡아야 아웃이었고, 1889년 이전까지는 볼 8~9개를 골라야 걸어나갈 수 있었다) 있었다고 한다. 즉, 타자들은 홈런 보다는 볼을 맞추는 데 집중하였고 그에 따라 타율이 타자의 기량을 나타나는 데 충분했다. 그러나 현대 야구에서는 '출루율', '장타율' 등이 더 의미있는 지표라 할 수 있다.</h3>

- <h3>출루율과 장타율로 비교해보자</h3>

<h2>출루율 비교</h2>
<h3>메이저리거</h3>

In [240]:
korean_majorleaguer_dict['강정호']

,연도,팀명,타율,경기,타수,득점,안타,2루타,3루타,홈런,루타,타점,도루,도루실패,볼넷,사구,삼진,병살타,실책
0,2006,현대,0.150,10,20,1,3,1,0,0,4,1,0,1,0,0,8,1,3
1,2007,현대,0.133,20,15,0,2,0,0,0,2,0,0,0,0,0,5,1,0
2,2008,우리,0.271,116,362,36,98,18,1,8,142,47,3,1,31,5,65,12,13
3,2009,히어로즈,0.286,133,476,73,136,33,2,23,242,81,3,2,45,4,81,18,15
4,2010,넥센,0.301,133,449,60,135,30,2,12,205,58,2,2,61,7,87,14,23
5,2011,넥센,0.282,123,444,53,125,22,2,9,178,63,4,6,43,9,62,12,13
6,2012,넥센,0.314,124,436,77,137,32,0,25,244,82,21,5,71,6,78,16,12
7,2013,넥센,0.291,126,450,67,131,21,1,22,220,96,15,8,68,6,109,18,15
8,2014,넥센,0.356,117,418,103,149,36,2,40,309,117,3,3,68,13,106,8,9
9,통산,통산,0.298,902,3070,470,916,193,10,139,1546,545,51,28,387,50,601,100,103


In [241]:
columns = ['연도', '안타', '볼넷', '사구', '타수']
player = korean_majorleaguer_dict['강정호'][columns]
player

,연도,안타,볼넷,사구,타수
0,2006,3,0,0,20
1,2007,2,0,0,15
2,2008,98,31,5,362
3,2009,136,45,4,476
4,2010,135,61,7,449
5,2011,125,43,9,444
6,2012,137,71,6,436
7,2013,131,68,6,450
8,2014,149,68,13,418
9,통산,916,387,50,3070


In [242]:
hitter_majorleaguer['Name']

0    강정호
1    김하성
2    김현수
3    박병호
4    이대호
5    최희섭
6    황재균
Name: Name, dtype: object

In [274]:
# 출루율 = (안타 + 볼넷 + 몸에맞는볼) / (타수 + 볼넷 + 몸에맞는볼 + 희생플라이)
# 희생플라이 데이터가 없다.... (일단 제외하고 계산)

fig = go.Figure()
columns = ['연도', '안타', '볼넷', '사구', '타수']

for idx, name in enumerate(hitter_majorleaguer['Name']):
    player = korean_majorleaguer_dict[name][columns]
    y_ = [round((int(player.iloc[idx]['안타'])+int(player.iloc[idx]['볼넷'])+int(player.iloc[idx]['사구'])) /
                               (int(player.iloc[idx]['타수'])+int(player.iloc[idx]['볼넷'])+int(player.iloc[idx]['사구'])), 3) for idx in range(player.shape[0]-1)]
    
    fig.add_trace(go.Scatter(x=player['연도'][:-1].astype(int),
                            y=y_,
                            hovertemplate=name+'(%{y})',
                            name=name))

fig.update_yaxes(showgrid=False, zeroline=False)
fig.update_xaxes(showgrid=False, zeroline=False)
fig.update_layout(title='메이저리거 출루율(희생플라이제외)',
                 plot_bgcolor='rgb(245, 245, 245)')
fig.show()

<h2>$출루율 = \frac{안타 + 볼넷 + 몸에맞는볼}{타수 + 볼넷 + 몸에맞는볼 + 희생플라이}$</h2>
<h3>하지만 희생플라이 데이터가 없어 제외시키고 출루율을 구하였더니, 2010년~2020년 동안 0.3에서 0.5사이에 출루율(희생플라이 제외)이 분포해 있는 것을 알 수 있다.</h3>

<h3>KBO</h3>

In [275]:
fig = go.Figure()

for name in top10_hitter_dict.keys():
    player = top10_hitter_dict[name][['연도', '출루율']]
    fig.add_trace(go.Scatter(x=player['연도'][:-1].astype(int),
                            y=player['출루율'][:-1].astype(float),
                            hovertemplate=name+'(%{y})',
                            name=name))

fig.update_yaxes(showgrid=False, zeroline=False)
fig.update_xaxes(showgrid=False, zeroline=False)
fig.update_layout(title='KBO top10 출루율',
                 plot_bgcolor='rgb(245, 245, 245)')
fig.show()

<h2>$출루율 = \frac{안타 + 볼넷 + 몸에맞는볼}{타수 + 볼넷 + 몸에맞는볼 + 희생플라이}$</h2>
<h3>2010년~2020년 동안 0.3에서 0.5 사이에 출루율이 분포하고 있다. 메이저리그 진출 선수들은 희생플라이 데이터를 제외하고 계산한 출루율이라 KBO top10 선수들의 출루율이 더 높은 것을 알 수 있다.</h3>

In [245]:
from sklearn.linear_model import LinearRegression

In [323]:
def compare_func(category):
    x_data = []
    y_data = []
    df = pd.DataFrame(columns=['year', 'category'])
    
    for name in hitter_majorleaguer['Name']:
        x_data.extend(korean_majorleaguer_dict[name].iloc[:-1]['연도'].astype(int).to_list())
        y_data.extend(korean_majorleaguer_dict[name].iloc[:-1][category].astype(float).to_list())
    
    for idx in range(len(x_data)):
        df.loc[idx] = [x_data[idx], y_data[idx]]
        
    X = df['year']
    y = df['category']
    
    line_fitter = LinearRegression()
    line_fitter.fit(X.values.reshape(-1,1), y)
    
    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(x=X, 
                              y=y, 
                              mode='markers',
                              name='메이저리거 선수들의 KBO시절 ' + category + ' 데이터',
                              marker_color='crimson'))
    fig1.add_trace(go.Scatter(x=X, 
                              y=line_fitter.predict(X.values.reshape(-1,1)),
                              mode='lines',
                              name='선형회귀 선',
                              marker_color='black'))
    
    fig1.update_yaxes(showgrid=False, zeroline=False)
    fig1.update_xaxes(showgrid=False, zeroline=False)
    fig1.update_layout(title='메이저리거 선수들의 KBO시절 ' + category + ' 선형회귀',
                      plot_bgcolor='rgb(245, 245, 245)')
    
    top10_hitter_player_name = []

    for name in top10_hitter_dict.keys():
        top10_hitter_player_name.append(name)

    rows_ = 2
    cols_ = 5
    i = 0

    fig2 = make_subplots(rows=rows_, cols=cols_)

    for r in range(1, rows_+1):
        for c in range(1, cols_+1):
            fig2.add_trace(go.Scatter(x=x_data,
                                    y=line_fitter.predict(X.values.reshape(-1,1)),
                                    mode='lines',
                                    marker_color='black',
                                    name='선형회귀 선',
                                    showlegend=False), r, c)
            fig2.add_trace(go.Scatter(x=top10_hitter_dict[top10_hitter_player_name[i]]['연도'][:-1].astype(int),
                                    y=top10_hitter_dict[top10_hitter_player_name[i]][category][:-1].astype(float),
                                    hovertemplate='%{x}년 (%{y})',
                                    mode='markers',
                                    marker_color='red',
                                    showlegend=False), r, c)
            fig2.update_yaxes(showgrid=False, zeroline=False, row=r, col=c)
            fig2.update_xaxes(showgrid=False, title_text=top10_hitter_player_name[i], row=r, col=c)
            i += 1

    fig2.update_layout(title='2020년 KBO top10 타자 ' + category + ' 데이터 vs 한국인 메이저리거',
                      plot_bgcolor='rgb(245, 245, 245)')
    fig1.show()
    fig2.show()

In [324]:
top10_hitter_dict['최형우'].columns

Index(['연도', '팀명', '타율', '경기', '타석', '타수', '득점', '안타', '2루타', '3루타', '홈런',
       '루타', '타점', '도루', '도루실패', '볼넷', '사구', '삼진', '병살타', '장타율', '출루율', '실책'],
      dtype='object')

In [325]:
korean_majorleaguer_dict['강정호'].columns

Index(['연도', '팀명', '타율', '경기', '타수', '득점', '안타', '2루타', '3루타', '홈런', '루타',
       '타점', '도루', '도루실패', '볼넷', '사구', '삼진', '병살타', '실책'],
      dtype='object')

In [326]:
# '연도', '팀명', '타율', '경기', '타수', '득점', '안타', '2루타', '3루타', '홈런', 
# '루타','타점', '도루', '도루실패', '볼넷', '사구', '삼진', '병살타', '실책'
#  insert above categories into compare_func()

compare_func('타율')

In [327]:
compare_func('득점')

In [331]:
compare_func('사구')

In [329]:
# 출루율 = (안타 + 볼넷 + 몸에맞는볼) / (타수 + 볼넷 + 몸에맞는볼 + 희생플라이)
# 희생플라이 데이터가 없다.... (일단 제외하고 계산)

x_data = []
y_data = []

fig = go.Figure()
columns = ['연도', '안타', '볼넷', '사구', '타수']

for idx, name in enumerate(hitter_majorleaguer['Name']):
    player = korean_majorleaguer_dict[name][columns]
    y_ = [round((int(player.iloc[idx]['안타'])+int(player.iloc[idx]['볼넷'])+int(player.iloc[idx]['사구'])) /
                               (int(player.iloc[idx]['타수'])+int(player.iloc[idx]['볼넷'])+int(player.iloc[idx]['사구'])), 3) for idx in range(player.shape[0]-1)]
    
    fig.add_trace(go.Scatter(x=player['연도'][:-1].astype(int),
                            y=y_,
                            hovertemplate=name+'(%{y})',
                            name=name))
    
    for i in range(0, player.shape[0]-1):
        x_data.append(int(player.iloc[i]['연도']))
        y_data.append(y_[i])

df = pd.DataFrame(columns=['year', 'on-base_rate'])

for idx in range(len(x_data)):
    df.loc[idx] = [x_data[idx], y_data[idx]]

X = df['year']
y = df['on-base_rate']

line_fitter = LinearRegression()
line_fitter.fit(X.values.reshape(-1, 1), y)

fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=X,
                        y=y,
                        mode='markers',
                        marker_color='crimson',
                        name='메이저리거 선수들의 KBO시절 출루율 데이터'))
fig1.add_trace(go.Scatter(x=X,
                        y=line_fitter.predict(X.values.reshape(-1, 1)),
                        mode='lines',
                        marker_color='black',
                        name='선형회귀 선'))
fig1.update_yaxes(showgrid=False, zeroline=False)
fig1.update_xaxes(showgrid=False, zeroline=False)
fig1.update_layout(title='메이저리거 선수들의 KBO시절 출루율 선형회귀',
                   plot_bgcolor='rgb(245, 245, 245)')
top10_hitter_player_name = []

for name in top10_hitter_dict.keys():
    top10_hitter_player_name.append(name)

rows_ = 2
cols_ = 5
i = 0

fig2 = make_subplots(rows=rows_, cols=cols_)

for r in range(1, rows_+1):
    for c in range(1, cols_+1):
        fig2.add_trace(go.Scatter(x=x_data,
                                y=line_fitter.predict(X.values.reshape(-1,1)),
                                mode='lines',
                                marker_color='black',
                                name='선형회귀 선',
                                showlegend=False), r, c)
        fig2.add_trace(go.Scatter(x=top10_hitter_dict[top10_hitter_player_name[i]]['연도'][:-1].astype(int),
                                y=top10_hitter_dict[top10_hitter_player_name[i]]['출루율'][:-1].astype(float),
                                hovertemplate='%{x}년 (%{y})',
                                mode='markers',
                                marker_color='red',
                                showlegend=False), r, c)
        fig2.update_yaxes(showgrid=False, zeroline=False, row=r, col=c)
        fig2.update_xaxes(showgrid=False, title_text=top10_hitter_player_name[i], row=r, col=c)
        i += 1

fig2.update_layout(title='2020년 KBO top10 타자 출루율 데이터 vs 한국인 메이저리거',
                  plot_bgcolor='rgb(245, 245, 245)')
fig1.show()
fig2.show()

In [330]:
# 출루율 = (안타 + 볼넷 + 몸에맞는볼) / (타수 + 볼넷 + 몸에맞는볼 + 희생플라이)
# 희생플라이 데이터가 없다.... (일단 제외하고 계산)

x_data = []
y_data = []

fig = go.Figure()
columns = ['연도', '안타', '2루타', '3루타', '홈런', '타수']

for idx, name in enumerate(hitter_majorleaguer['Name']):
    player = korean_majorleaguer_dict[name][columns]
    y_ = [(int(player.iloc[idx]['안타'])+int(player.iloc[idx]['2루타'])*2+int(player.iloc[idx]['3루타'])*3+ int(player.iloc[idx]['홈런'])*4)/
                               int(player.iloc[idx]['타수']) for idx in range(player.shape[0]-1)]
    
    fig.add_trace(go.Scatter(x=player['연도'][:-1].astype(int),
                            y=y_,
                            hovertemplate=name+'(%{y})',
                            name=name))
    
    for i in range(0, player.shape[0]-1):
        x_data.append(int(player.iloc[i]['연도']))
        y_data.append(y_[i])

df = pd.DataFrame(columns=['year', 'slugging_percentage'])

for idx in range(len(x_data)):
    df.loc[idx] = [x_data[idx], y_data[idx]]

X = df['year']
y = df['slugging_percentage']

line_fitter = LinearRegression()
line_fitter.fit(X.values.reshape(-1, 1), y)

fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=X,
                        y=y,
                        mode='markers',
                        marker_color='crimson',
                        name='메이저리거 선수들의 KBO시절 장타율 데이터'))
fig1.add_trace(go.Scatter(x=X,
                        y=line_fitter.predict(X.values.reshape(-1, 1)),
                        mode='lines',
                        marker_color='black',
                        name='선형회귀 선'))
fig1.update_yaxes(showgrid=False, zeroline=False)
fig1.update_xaxes(showgrid=False, zeroline=False)
fig1.update_layout(title='메이저리거 선수들의 KBO시절 장타율 선형회귀',
                   plot_bgcolor='rgb(245, 245, 245)')
top10_hitter_player_name = []

for name in top10_hitter_dict.keys():
    top10_hitter_player_name.append(name)

rows_ = 2
cols_ = 5
i = 0

fig2 = make_subplots(rows=rows_, cols=cols_)

for r in range(1, rows_+1):
    for c in range(1, cols_+1):
        fig2.add_trace(go.Scatter(x=x_data,
                                y=line_fitter.predict(X.values.reshape(-1,1)),
                                mode='lines',
                                marker_color='black',
                                name='선형회귀 선',
                                showlegend=False), r, c)
        fig2.add_trace(go.Scatter(x=top10_hitter_dict[top10_hitter_player_name[i]]['연도'][:-1].astype(int),
                                y=top10_hitter_dict[top10_hitter_player_name[i]]['장타율'][:-1].astype(float),
                                hovertemplate='%{x}년 (%{y})',
                                mode='markers',
                                marker_color='red',
                                showlegend=False), r, c)
        fig2.update_yaxes(showgrid=False, zeroline=False, row=r, col=c)
        fig2.update_xaxes(showgrid=False, title_text=top10_hitter_player_name[i], row=r, col=c)
        i += 1

fig2.update_layout(title='2020년 KBO top10 타자 장타율 데이터 vs 한국인 메이저리거',
                  plot_bgcolor='rgb(245, 245, 245)')
fig1.show()
fig2.show()